In [6]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     -- ------------------------------------- 0.3/4.1 MB ? eta -:--:--
     --------------------------- ------------ 2.9/4.1 MB 12.9 MB/s eta 0:00:01
     ---------------------------------------- 4.1/4.1 MB 12.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     -- ------------------------------------- 0.3/4.1 MB ? eta -:--:--
     ------------------------------ --------- 3.1/4.1 MB 13.2 MB/s eta 0:00:01
     ---------------------------------------- 4.1/4.1 MB 11.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     -- ------------------------------------- 0.3/4.1 MB ? eta -:--:--
     ------------------------------ --------- 3.1/4.

  You can safely remove it manually.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
import time

In [6]:
# Configuración de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# Transformaciones para normalizar imágenes (Requeridas por ResNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar imágenes
    transforms.ToTensor(),  # Convertir a tensores
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización de ResNet
])

In [10]:
# Rutas a los conjuntos de entrenamiento y validación
train_dir = "./Data/train"
valid_dir = "./Data/valid"

In [11]:
# Cargar imágenes desde carpetas (PyTorch usa estructura basada en subcarpetas para cada clase)
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)

In [12]:
# Definir DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [14]:
# Verificar categorías detectadas
print("Clases detectadas:", train_dataset.classes)

Clases detectadas: ['freshripe', 'freshunripe', 'overripe', 'ripe', 'rotten', 'unripe']


In [15]:
# Cargar modelo ResNet-18 preentrenado
num_classes = 6  # Tenemos 6 categorías
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modificar capa final para 6 clases
model = model.to(device)

c:\Users\Raxie\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Raxie\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Raxie/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 85.1MB/s]


In [16]:
# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [17]:
# Entrenamiento del modelo
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    start_time = time.time()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Acc: {acc:.2f}%, Time: {time.time() - start_time:.2f}s")

Epoch 1/10, Loss: 0.3719, Acc: 87.34%, Time: 131.21s
Epoch 2/10, Loss: 0.1878, Acc: 93.43%, Time: 121.63s
Epoch 3/10, Loss: 0.1721, Acc: 94.17%, Time: 122.05s
Epoch 4/10, Loss: 0.1435, Acc: 95.24%, Time: 118.04s
Epoch 5/10, Loss: 0.0938, Acc: 96.93%, Time: 119.10s
Epoch 6/10, Loss: 0.0750, Acc: 97.46%, Time: 136.66s
Epoch 7/10, Loss: 0.0749, Acc: 97.40%, Time: 136.62s
Epoch 8/10, Loss: 0.0703, Acc: 97.46%, Time: 147.11s
Epoch 9/10, Loss: 0.0681, Acc: 97.44%, Time: 151.11s
Epoch 10/10, Loss: 0.0493, Acc: 98.53%, Time: 148.51s


In [18]:
# Evaluación del modelo en validación
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
val_acc = 100 * correct / total
print(f"Accuracy en validación: {val_acc:.2f}%")

Accuracy en validación: 94.66%


In [19]:
# Guardar el modelo entrenado
torch.save(model.state_dict(), "Modelo_MadurezPlatano.pth")
print("Modelo guardado como 'Modelo_MadurezPlatano.pth'")

Modelo guardado como 'Modelo_MadurezPlatano.pth'


In [2]:
! pip install cuda-python

In [8]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [2]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Fri_Feb_21_20:42:46_Pacific_Standard_Time_2025
Cuda compilation tools, release 12.8, V12.8.93
Build cuda_12.8.r12.8/compiler.35583870_0


In [3]:
! python -c "import torch; print(torch.cuda.is_available())"

OMP: Error #15: Initializing libiomp5md.dll, but found libiomp5md.dll already initialized.
OMP: Hint This means that multiple copies of the OpenMP runtime have been linked into the program. That is dangerous, since it can degrade performance or cause incorrect results. The best thing to do is to ensure that only a single OpenMP runtime is linked into the process, e.g. by avoiding static linking of the OpenMP runtime in any library. As an unsafe, unsupported, undocumented workaround you can set the environment variable KMP_DUPLICATE_LIB_OK=TRUE to allow the program to continue to execute, but that may cause crashes or silently produce incorrect results. For more information, please see http://www.intel.com/software/products/support/.


In [4]:
import torch
print("CUDA disponible:", torch.cuda.is_available())
print("Versión de CUDA:", torch.version.cuda)
print("Dispositivos CUDA:", torch.cuda.device_count())
print("Nombre de la GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No detectada")

: 